In [ ]:
import concurrent.futures
import os
from urllib.parse import urlparse
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import requests
from tqdm import tqdm

In [ ]:
headers = {
    'User-Agent': 'Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)'
}

chrome_options = ChromeOptions()

session = requests.Session()

def process_product_page(product_page):
    try:
        product_url = product_page['href']

        driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=chrome_options)

        driver.get(product_url)
        driver.implicitly_wait(5)

        driver.find_element(By.CSS_SELECTOR, 'div._3osy73V_eD._1Hc_ju_IXp > button').click()
        driver.implicitly_wait(5)

        full_html = driver.page_source

        driver.quit()

        soup = BeautifulSoup(full_html, 'html.parser')

        product_name = soup.select_one('h3._22kNQuEXmb._copyable').text

        valid_product_name = ''.join(c for c in os.path.splitext(product_name)[0] if c.isalnum() or c in (' ', '_')).rstrip()
        valid_product_name = valid_product_name.replace(' ', '_')
        target_directory = os.path.abspath(f'naver/{valid_product_name}')

        os.makedirs(target_directory, exist_ok=True)

        image_tags = soup.select('.se-main-container img')

        for i, img in enumerate(image_tags):
            if 'data-src' in img.attrs and img['data-src'].strip():
                image_url = img['data-src']

                parsed_url = urlparse(image_url)
                filename, extension = os.path.splitext(os.path.basename(parsed_url.path))

                file_path = os.path.join(target_directory, f'image_{i}{extension}')

                response = session.get(image_url)

                with open(file_path, 'wb') as f:
                    f.write(response.content)

    except Exception as e:
        pass

with concurrent.futures.ThreadPoolExecutor() as executor:
    for i in tqdm(range(1, 20714), desc='Processing pages'):
        url = f'https://search.shopping.naver.com/search/all?adQuery=%EC%82%AC%EB%A3%8C&frm=NVSHATC&npayType=2&origQuery=%EC%82%AC%EB%A3%8C&pagingIndex={i}&pagingSize=40&productSet=checkout&query=%EC%82%AC%EB%A3%8C&sort=rel&timestamp=&viewType=image'

        response = session.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')

        product_pages = soup.select('div.thumbnail_thumb_wrap__RbcYO._wrapper > a')

        executor.map(process_product_page, product_pages)


KeyboardInterrupt

